In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://127.0.0.1:27017/")

In [3]:
client.list_database_names()

['admin', 'config', 'knowledgehut', 'local', 'newdb']

In [4]:
db = client['knowledgehut']

In [5]:
type(db)

pymongo.database.Database

In [6]:
car_collection = db['cars']

In [7]:
type(car_collection)

pymongo.collection.Collection

In [8]:
cursor = car_collection.find({})

In [9]:
type(cursor)

pymongo.cursor.Cursor

In [10]:
for doc in cursor:
    print(doc)

{'_id': ObjectId('64d3881268d05709c6387353'), 'Name': 'chevrolet chevelle malibu', 'Miles_per_Gallon': 18, 'Cylinders': 8, 'Displacement': 307, 'Horsepower': 130, 'Weight_in_lbs': 3504, 'Acceleration': 12, 'Year': '1970-01-01', 'Origin': 'USA'}
{'_id': ObjectId('64d3881268d05709c6387354'), 'Name': 'buick skylark 320', 'Miles_per_Gallon': 15, 'Cylinders': 8, 'Displacement': 350, 'Horsepower': 165, 'Weight_in_lbs': 3693, 'Acceleration': 11.5, 'Year': '1970-01-01', 'Origin': 'USA'}
{'_id': ObjectId('64d3881268d05709c6387355'), 'Name': 'plymouth satellite', 'Miles_per_Gallon': 18, 'Cylinders': 8, 'Displacement': 318, 'Horsepower': 150, 'Weight_in_lbs': 3436, 'Acceleration': 11, 'Year': '1970-01-01', 'Origin': 'USA'}
{'_id': ObjectId('64d3881268d05709c6387356'), 'Name': 'amc rebel sst', 'Miles_per_Gallon': 16, 'Cylinders': 8, 'Displacement': 304, 'Horsepower': 150, 'Weight_in_lbs': 3433, 'Acceleration': 12, 'Year': '1970-01-01', 'Origin': 'USA'}
{'_id': ObjectId('64d3881268d05709c6387357'),

In [12]:
car_collection.create_index('Miles_per_Gallon')

'Miles_per_Gallon_1'

In [13]:
car_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'Miles_per_Gallon_1': {'v': 2, 'key': [('Miles_per_Gallon', 1)]}}

In [16]:
cursor = car_collection.find({'Cylinders': {'$eq': 8}})

In [17]:
len(list(cursor))

108

In [18]:
car_collection.find({'Cylinders': {'$eq': 8}}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'knowledgehut.cars',
  'indexFilterSet': False,
  'parsedQuery': {'Cylinders': {'$eq': 8}},
  'queryHash': 'B5C8E7D6',
  'planCacheKey': 'B5C8E7D6',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'Cylinders': {'$eq': 8}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 108,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 406,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'Cylinders': {'$eq': 8}},
   'nReturned': 108,
   'executionTimeMillisEstimate': 0,
   'works': 407,
   'advanced': 108,
   'needTime': 298,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 406},
  'allPlansExecution': []},
 'command': {'find': 'cars',
  'filter': {'C

In [19]:
car_collection.count_documents({})

406

In [20]:
car_collection.find({'Miles_per_Gallon': {'$gt': 20}}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'knowledgehut.cars',
  'indexFilterSet': False,
  'parsedQuery': {'Miles_per_Gallon': {'$gt': 20}},
  'queryHash': '6C7E9E49',
  'planCacheKey': 'ACE5B64F',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'Miles_per_Gallon': 1},
    'indexName': 'Miles_per_Gallon_1',
    'isMultiKey': False,
    'multiKeyPaths': {'Miles_per_Gallon': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'Miles_per_Gallon': ['(20, inf.0]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 238,
  'executionTimeMillis': 1,
  'totalKeysExamined': 238,
  'totalDocsExamined': 238,
  'executionStages': {'stage': 'FETCH',
   'nReturned': 238,
   'executionTimeMillisEs

In [21]:
cursor = car_collection.aggregate([])

In [23]:
len(list(cursor))

406

In [24]:
cursor = car_collection.aggregate([
    # Stage 1
    {"$match":{'Miles_per_Gallon':{"$gt": 20}}}
])

In [26]:
len(list(cursor))

238

In [32]:
cursor = car_collection.aggregate([
    # Stage 1
    {"$match":{'Miles_per_Gallon':{"$gt": 20}}},
    # Stage 2
    {"$group": {'_id': "$Cylinders"}}
])

In [34]:
for doc in cursor:
    print(doc)

{'_id': 6}
{'_id': 5}
{'_id': 4}
{'_id': 8}
{'_id': 3}


In [40]:
cursor = car_collection.aggregate([
    # Stage 1
    {"$match":{'Miles_per_Gallon':{"$gt": 20}}},
    # Stage 2
    {
        "$group":{
            "_id": {
                'Cylinders': '$Cylinders'
            },
            "avg_mileage": {
                '$avg': '$Miles_per_Gallon'
            },
            "count_doc": {
                '$sum': 1
            }
        }
    }
])

In [41]:
for doc in cursor:
    print(doc)

{'_id': {'Cylinders': 5}, 'avg_mileage': 27.366666666666664, 'count_doc': 3}
{'_id': {'Cylinders': 3}, 'avg_mileage': 22.6, 'count_doc': 2}
{'_id': {'Cylinders': 4}, 'avg_mileage': 29.691326530612244, 'count_doc': 196}
{'_id': {'Cylinders': 8}, 'avg_mileage': 23.425, 'count_doc': 4}
{'_id': {'Cylinders': 6}, 'avg_mileage': 23.26060606060606, 'count_doc': 33}


In [42]:
cursor = car_collection.aggregate([
    # Stage 1
    {"$match":{'Miles_per_Gallon':{"$gt": 20}}},
    # Stage 2
    {
        "$group":{
            "_id": {
                'Cylinders': '$Cylinders'
            },
            "avg_mileage": {
                '$avg': '$Miles_per_Gallon'
            },
            "count_doc": {
                '$sum': 1
            }
        }
    },
    # Stage 3
    {
        "$sort":{
            "avg_mileage": -1
        }
    }
])

In [43]:
for doc in cursor:
    print(doc)

{'_id': {'Cylinders': 4}, 'avg_mileage': 29.691326530612244, 'count_doc': 196}
{'_id': {'Cylinders': 5}, 'avg_mileage': 27.366666666666664, 'count_doc': 3}
{'_id': {'Cylinders': 8}, 'avg_mileage': 23.425, 'count_doc': 4}
{'_id': {'Cylinders': 6}, 'avg_mileage': 23.26060606060606, 'count_doc': 33}
{'_id': {'Cylinders': 3}, 'avg_mileage': 22.6, 'count_doc': 2}


In [44]:
cursor = car_collection.aggregate([
    # Stage 1
    {"$match":{'Miles_per_Gallon':{"$gt": 20}}},
    # Stage 2
    {
        "$group":{
            "_id": {
                'Cylinders': '$Cylinders'
            },
            "avg_mileage": {
                '$avg': '$Miles_per_Gallon'
            },
            "count_doc": {
                '$sum': 1
            }
        }
    },
    # Stage 3
    {
        "$sort":{
            "avg_mileage": -1
        }
    },
    # Stage 4
    {
        "$limit": 3
    }
])

In [45]:
for doc in cursor:
    print(doc)

{'_id': {'Cylinders': 4}, 'avg_mileage': 29.691326530612244, 'count_doc': 196}
{'_id': {'Cylinders': 5}, 'avg_mileage': 27.366666666666664, 'count_doc': 3}
{'_id': {'Cylinders': 8}, 'avg_mileage': 23.425, 'count_doc': 4}
